In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

# There's a lot of columns in the DF. 
# Therefore, we add this option so that we can see more columns
pd.options.display.max_columns = 100

# Data Preprocessing

In this notebook, we will perform some cleaning and analysis of the data before applying the Machine Learning algorithm. 

First, let us load the data.

In [ ]:
df = pd.read_csv('./Data/CrowdstormingDataJuly1st.csv')
df.head(10)

The data description, which is the `Data.md` file in the `Data` folder, tells us that we are working on 
> a dataset of player–referee dyads including the number of matches players and referees encountered each other and our dependent variable, the number of red cards given to a player by a particular referee throughout all matches the two encountered each other.

A dyad between a player and a referee simply means the interaction between a player and a referee. 


## 1. Cleaning the NaNs values for the colours of players

We want to see the number of entries in the Dataframe before starting to clean the DataFrame.

In [ ]:
df_size_init = df.shape
print(df_size_init)

The first step we want to take is to remove all the players from which the skin column was not identified, i.e. the ones who have NaN entries in the *rater1* and *rater2* columns, which measures the darkness of the skin color.

In [ ]:
df = df.dropna(subset=['rater1', 'rater2'])
print(df.shape)

We see now that we dropped already quite some entries.

In [ ]:
df_size_init[0] - df.shape[0]

In [ ]:
# Check for other null values
df.isnull().any()

There are still quite few entries that have null values, and we want to assess if they are correlated in any fashion.

In [ ]:
print(df[df['meanIAT'].isnull()].shape)
print(df[df['meanExp'].isnull()].shape)

We see first that there is exactly the same number of entries having NaN *meanIAT* and *meanExp* entries, and we see below that

In [ ]:
df[df['meanExp'].isnull()].refNum.unique()

We see that there are only a few referees (the names are not here for anonymization purposes), that have NaN entries in both field. Let's just remove the them from our dataset.

In [ ]:
df = df[~df['meanIAT'].isnull()]
print(df.isnull().any())
print(df.shape)

We can see here that there are still some columns with NaN entries, which we will deal with later on.

## 2. Uniquely defining the skin column : creating a `skin_colour` column

We define the skin_colour column as the mean of the *rater1* and *rater2* columns. They are described as the following  :

>rater1 - skin rating of photo by rater 1 (5-point scale ranging from “very light skin” to “very dark skin”)

>rater2 - skin rating of photo by rater 2 (5-point scale ranging from “very light skin” to “very dark skin”).

As both describe the same thing with different scales, it is sensible to average them. We then remove the *rater1* and *rater2* columns, as they will not be of use for us anymore. 

In [ ]:
df['skin_colour'] = df[['rater1', 'rater2']].mean(axis=1)
df = df.drop(['rater1', 'rater2'], axis=1)
df.head()

Let us quickly check all the different `skin_colour` values that a player can have.

In [ ]:
unique_colours = df['skin_colour'].unique()
np.sort(unique_colours)

We see the 5 values that were initially given by the *rater1* and *rater2* columns, along with the values inbetween those, leading us to a grand total of 9 different entries (still from “very light skin” to “very dark skin”)

## 3. Removing some more features

There are many columns that do not convey useful information to our purpose, we want to identify and remove them.

In [ ]:
print(df.columns)

There are some very obvious entries that are of no use to us : 
- *photoID*: We do not have the pictures !
- *refNum*: We will aggregate the values by the name of the player, so we do not need it.
- *refCountry*: We do not need it for the same reason.
- *Alpha_3*: We do not need it for the same reason.

The following features only are related to a referee, and we will remove them as well.
- *meanIAT*
- *nIAT*
- *seIAT*
- *meanExp*
- *nExp*
- *seExp*

**N.B.** We are doing this as the aim of the exercise is to determine the skin colour of a player from the stats we have on him. His skin colour should not depend on the referee present in his matches ;)

In [ ]:
columns_to_remove = ['photoID', 'refNum', 'refCountry', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT', 'meanExp', 'nExp', 'seExp']
df_cleaned = df.drop(columns_to_remove, axis=1)

In [ ]:
print(df_cleaned.shape)
df_cleaned.head(10)

We did not change the number of entries here, but now only have 17 columns remaining. The following transformation is to get all the data related to each player, i.e. aggregate by their name.

## 4. Aggregating the table

The number of unique `player` entries is our database is way smaller than the total number of entries !

In [ ]:
print("Total number of players: ", len(df_cleaned['player'].unique()))

This means that have extensive data on a few players, as they played many matches in their career. We will now perform the aggregation of our data.

In [ ]:
unique_players = df_cleaned['player'].unique()

Let us create the DataFrame in which we'll store our unique player entries. 

In [ ]:
# Values on whichc we want to sum
summed = ['games', 'victories', 'ties', 'defeats', 'goals', 
         'yellowCards', 'yellowReds', 'redCards']
#Dirty trick to keep the NaNs in the groupby because there is no other was to do so (opened issue in the project)
df_players = df_cleaned.replace(np.nan, -1000).groupby(['playerShort', 'player' ,'club' ,'leagueCountry', 'birthday', 'height', 'weight','position','skin_colour'])
df_players = df_players[summed].sum()
df_players = df_players.reset_index().replace(-1000, np.NaN)
print(df_players.shape)

df_players_no_nan = df_players.dropna()
print(df_players_no_nan.shape)

df_players.head()

We see here that if we don't keep the NaNs, we lose many players in the process. We want to see whether it is justified to do so. We computed all the players which are exluded of our newer DataFrame. First of all, we just want to note that that there are a few players who have the same name, so the sorting by the *playerShort* attribute will yield unique results. One such example is the following :

In [ ]:
#Check whether the entries are unique for the playerShort attribute
print(len(df_players.playerShort) - len(df_players.playerShort.unique()))

#Check whether the entries are unique for the player attribute
print(len(df_players.player) - len(df_players.player.unique()))

df_players.loc[df_players.player == ' Juanfran']

Let us now focus on the players which were excluded from the aggregation. There are three distinct cases for the exclusion of players from the DataFrame, as we show below. Either
- the player has no *weight* entry,
- the player has no *position* entry,
- the player has no *height* entry,
- or it can be a combination of those. 

Let us show a player with no *position entry*.

Here we have a player with both no *weight*, no *height* and no *position* !

In [ ]:
df_players.loc[df_players['height'].isnull()]

We see that removing them is the simplest approach we can take, but let us first check the percentage of the population that they represent. 

In [ ]:
print("Percentage of people removed with the aggregation: ", 100*(1-df_players_no_nan.shape[0]/len(df_cleaned.playerShort.unique())), "%")

Even if they represent 10% of our population, we can remove them as including them in the Machine Learning process would yield many complications, having to create special cases for each of the missing data. We will however keep them in a separate DataFrame for comparison's sake.

We also discretise the country in which they're playing and the birthYear in order to be able to use it in our algrithm, as it only takes numeric values.

In [ ]:
df_players['english'] = [int(country == 'England') for country in df_players['leagueCountry']]
df_players['german'] = [int(country == 'Germany') for country in df_players['leagueCountry']]
df_players['french'] = [int(country == 'France') for country in df_players['leagueCountry']]
df_players['spanish'] = [int(country == 'Spain') for country in df_players['leagueCountry']]
df_players['birthYear'] = [int(b[-4:]) for b in df_players['birthday']]


We now save these DataFrames, making sure before that there are no more NaNs in our data.

In [ ]:
print(df_players_no_nan.isnull().any())

#actualise the value of the DataFrame
df_players_no_nan = df_players.dropna()

df_players.to_csv('./Data/players.csv', index=False)
df_players_no_nan.to_csv('./Data/players_no_nan.csv', index=False)

We will also save the dataFrames with only the numeric features.

In [ ]:
non_numeric = ['playerShort', 'player', 'club', 'leagueCountry', 'birthday', 'position']
df_players.drop(non_numeric, axis=1).to_csv('./Data/players_num_values.csv', index=False)
df_players_no_nan.drop(non_numeric, axis=1).to_csv('./Data/players_no_nan_only_num_values.csv', index=False)

## 5. Cross-Featuring

We can introduce some cross-features with the features we removed. We know for example that the columns with
- **IAT** refer to the **race IAT test**, *i.e.* a high value correspond to someone who will prefer a white person (thinks it's a better person) over a black person.
- **Exp** refer to the use of a **racial thermometer task**, *i.e.* a high value corresponds to greater feelings of warmth toward whites versus blacks.

If we want to keep these information, we can cross-feature those information with the yellow and red cards given to the player . 

Let's introduce a **new variable**. We will call it the `gravity_factor`. It is given by:


\begin{equation}
G = Y + \frac{\sum Y}{\sum YR}  YR + \frac{\sum Y }{\sum R} R
\end{equation}
where:
- $Y$ is the number of yellow cards given to a player
- $\sum Y$ is the total number of yellow cards given
- $YR$ is the number of yellow then red cards given to a player
- $\sum YR$ is the total number of yellow then red cards given
- $R$ is the number of red cards given to a player
- $\sum R$ is the total number of red cards given


So, let's add this new value to the big DF.

In [ ]:
print("Percentage of yellow cards: ", 100*df['yellowCards'].sum()/
      df['games'].sum(), "%")
print("Percentage of yellow then red cards: ", 100*df['yellowReds'].sum()/
      df['games'].sum(), "%")
print("Percentage of red cards: ", 100*df['redCards'].sum()/
      df['games'].sum(), "%")

In [ ]:
nbr_yellow = df['yellowCards'].sum()
nbr_yellowRed = df['yellowReds'].sum()
nbr_red = df['redCards'].sum()

In [ ]:
gravity = df['yellowCards'] + nbr_yellow/nbr_yellowRed * df['yellowReds'] + nbr_yellow/nbr_red * df['redCards']
gravity.unique()

In [ ]:
df_with_features = df
df_with_features['gravity'] = gravity
df_with_features.head()

In [ ]:
gravity_players = df_players['yellowCards'] + nbr_yellow/nbr_yellowRed * df_players['yellowReds'] + nbr_yellow/nbr_red * df_players['redCards']

In [ ]:
df_players_with_features = df_players
df_players_with_features['gravity'] = gravity_players
df_players_with_features.head()

In [ ]:
# Just check that the aggregation is correct
print("Gravity for 1st player in aggregated table: ", df_players_with_features['gravity'][0])
print("Gravity for 1st player in aggregated table: ", 
      df_with_features[df_with_features['player'] == 'Lucas Wilchez']['gravity'].sum())

# It's ok.. =)

We can now introduce another cross-feature. The purpose of this one is to use the *meanIAT* and *meanExp* values. To do so, we will link them with the 4 following features: *yellowCards*, *yellowReds*, *redCards*, and *gravity*. We will use the following equation:

\begin{equation}
F_{I-C} (P) = \frac{1}{\#R(P)} \sum_{r\in R(P)} \frac{C(r)}{E(r)} \cdot I(r)
\end{equation}

where:
- $F_{I-C} (P)$ defines the new feature.
- $I$ is the indicator. It can be *meanIAT* or *meanExp*
- $C$ defines the types of card received. It can be *yellowCards*, *yellowReds*, *redCards*, or *gravity*
- $P$ is a given player.
- $\#R(P)$ is the number of referees with whom a player played.
- $\sum_{r\in R(P)}$ is the sum over all referees who played with a given player.
- $C(r)$ is the value of $C$ for a specific Player and a specific referee.
- $E(r)$ is the number of times a referee played with a player.
- $I(r)$ is the indicator of the referee.

This will create 8 new features.

In [ ]:
indicators = ['meanIAT', 'meanExp']
cards = ['yellowCards', 'yellowReds', 'redCards', 'gravity']

In [ ]:
for idcts in indicators:
    for crds in cards:
        feature_name = idcts + "_" + crds
        array_feature = []
        print("Start feature ", feature_name)
        for plyr in df_players['player']:
            feature = 0
            # Get the DF with the player
            df_player = df[df['player'] == plyr]
            # Number of different referee
            nbr_ref = len(df_player)
            # Reindex
            df_player.index = np.arange(nbr_ref)
            # Sum for the new feature
            for i in range(nbr_ref):
                feature += df_player[crds][i]/df_player['games'][i]*df_player[idcts][i]
            # Add it into the array
            array_feature.append(feature/nbr_ref)
        # Add it into the DataFrame
        df_players_with_features[feature_name] = array_feature
df_players_with_features.head()

In [ ]:
# Let's save the new DF
df_players_with_features.to_csv('./Data/players_with features.csv', index=False)

In [ ]:
# We can also save the one without the nan
df_players_with_features_no_nan = df_players_with_features.dropna()
df_players_with_features_no_nan.to_csv('./Data/players_with features_no_nan.csv', index=False)

In [ ]:
# We can remove all the columns with String values instead of numerical values.
non_numeric = ['playerShort', 'player', 'club', 'leagueCountry', 'birthday', 'position']
df_players_with_features.drop(non_numeric, axis=1).to_csv('./Data/players_with features_only_num_values.csv', index=False)
df_players_with_features_no_nan.drop(non_numeric, axis=1).to_csv('./Data/players_with features_no_nan_only_num_values.csv', index=False)

## Visualization of the data

Let's do some scatter plot to see if some features are interesting with the *skin_colour*. We expect that some features such as the number of games played will be irrelevant with the *skin_colour*. 

In [ ]:
df_players_with_features_no_nan.shape

In [ ]:
features_tmp = df_players_with_features.columns
features = []
label = 'skin_colour'
for i in features_tmp:
    if i != 'skin_colour':
        features.append(i)
len(features)

In [ ]:
for ftre in features:
    if ftre not in non_numeric:
        plt.figure()
        plt.scatter(list(df_players_with_features_no_nan[ftre]), list(df_players_with_features_no_nan[label]))
        plt.title('Feature: %s'%(ftre))
        plt.xlabel(ftre)
        plt.ylabel(label)
        plt.show()

In [ ]:
# Just plot the values of meanExp because some values are less than 0
plt.figure()
plt.plot(df['meanExp'],color='b')
plt.plot(df['meanIAT'],color='g')

Plotting the data that are available did not seem to exhibit any obvious characteristic that would be correlated to the skin colour of a player. We will see whether our machine learning algorithm will be able to find a good classification.